# 05: Governance Assumption Analysis

Interrogates the assumptions embedded in anomaly detection systems. Every model that defines 'normal' also defines what deviation is punished. Both insider threat and agent monitoring encode the same governance assumptions.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown

from src.data.trail_loader import load_trail_dataset, get_trail_labels
from src.data.trace_loader import load_trace_dataset, trace_to_otel_format
from src.features.agent_extractor import AgentTraceFeatureExtractor
from src.features.ubfs_schema import UBFSNormalizer, ubfs_feature_names
from src.governance.assumption_audit import (
    audit_normality_assumptions,
    compare_baseline_distributions,
    governance_report,
)

## Normality Assumptions

In [ ]:
assumptions = audit_normality_assumptions()
for a in assumptions:
    print(f'\n--- {a.feature_name} ---')
    print(f'  CERT:   {a.cert_assumption}')
    print(f'  Agent:  {a.agent_assumption}')
    print(f'  Impact: {a.governance_implication}')

## Distribution Comparison

In [ ]:
trail = load_trail_dataset()
ext1 = AgentTraceFeatureExtractor()
X_trail, _, _ = ext1.extract_batch(trail['traces'])
X_trail = UBFSNormalizer('zscore').fit_transform(X_trail)

trace_data = load_trace_dataset()
otel = [trace_to_otel_format(t) for t in trace_data['trajectories']]
ext2 = AgentTraceFeatureExtractor()
X_trace, _, _ = ext2.extract_batch(otel)
X_trace = UBFSNormalizer('zscore').fit_transform(X_trace)

comparison = compare_baseline_distributions(X_trail, X_trace)
names = ubfs_feature_names()

print(f'{"Feature":30s} {"KL Divergence":>15s}')
print('-' * 50)
for feat in comparison:
    kl = feat.get('kl_divergence', 0)
    print(f'{feat["feature"]:30s} {kl:15.4f}')

## Governance Report

In [ ]:
report = governance_report(assumptions, comparison)
Markdown(report)